<a href="https://colab.research.google.com/github/techfreakworm/MovieReview-SentimentAnalysis/blob/tensorflow-dnn/SentimentAnalysisUsingMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from keras.datasets import imdb

In [0]:
# vocabulary_size = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
# print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

In [0]:
# type(X_train)

In [0]:
# X_train

In [0]:
# # Install the latest Tensorflow version.
# !pip install --quiet "tensorflow>=1.7"
# # Install TF-Hub.
# !pip install -q tensorflow-hub
# !pip install -q seaborn

In [0]:
# !ls sample_data

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

,sentence,sentiment,polarity
0,I saw the 10p.m. showing and I must say that t...,1,0
1,"""Gespenster"" Question of to be cool in the Ger...",8,1
2,"Charming doesn't even begin to describe ""Savin...",8,1
3,"Its not the cast. A finer group of actors, you...",4,0
4,I just read a review defending this film becau...,2,0


In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
sentence     25000 non-null object
sentiment    25000 non-null object
polarity     25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [0]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, test_df["polarity"], shuffle=False)

In [0]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
    trainable=True)

In [0]:
#Training config added later (required for saving estimator)
# training_config = tf.estimator.RunConfig(
#         model_dir=FLAGS.model_dir,
#         save_summary_steps=100,
#         save_checkpoints_steps=100)

In [0]:
estimator = tf.estimator.DNNClassifier(
#     config=training_config,
    hidden_units=[500, 1000, 1000, 500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))

In [0]:
# Training for 1,000 steps means 128,000 training examples with the default
# batch size. This is roughly equivalent to 5 epochs since the training dataset
# contains 25,000 examples.
estimator.train(input_fn=train_input_fn, steps=1000);

In [0]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

In [0]:
print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

Training set accuracy: 0.9721199870109558
Test set accuracy: 0.868399977684021


# For Server Input function (Defines how inputs would be handled when the inputs are exported and served)

In [0]:
# def serving_input_receiver_fn():
#     """
#     This is used to define inputs to serve the model.
#     :return: ServingInputReciever
#     """
#     receiver_tensors = {
#         'sentence': tf.placeholder(tf.string, [None, 1])
#     }

#     # Convert give inputs to adjust to the model.
#     features = {
#         'sentence': tf.concat([
#             receiver_tensors['sentence']
#         ], axis=1)
#     }
#     return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
#                                                     features=features)


In [0]:
# feature_spec = tf.feature_column.make_parse_example_spec(embedded_text_feature_column)
# export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

In [0]:
# embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
# embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [0]:
sentence_array = np.array(['My friends criticised this movie a lot, i thought this would be a complete shit show, and they were actually right. This movie didn\'t have good concept, no clear storyline and not even a good ending'])
features = {
'sentence': sentence_array
}

In [0]:
prediction_function = tf.estimator.inputs.numpy_input_fn(x=features, batch_size=len(features), shuffle=False)

In [0]:
predictions = list(estimator.predict(input_fn=prediction_function))

In [0]:
predictions

[{'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.39732113], dtype=float32),
  'logits': array([-0.41663954], dtype=float32),
  'probabilities': array([0.60267884, 0.39732116], dtype=float32)}]

In [0]:
predictions[0]['class_ids'][0]

0

In [0]:
!ls models

ckpt
